In [ ]:
import pandas as pd
import os
import sqlite3

#base de datos
ruta = os.getcwd()
pathdb=(f"{ruta}/zoo_database0.db")
conexion = sqlite3.connect(pathdb)
cursor = conexion.cursor()

pathx=(f"{ruta}/datos.xlsx")
path=(f"{ruta}/datos.json")
dfx = pd.read_excel(pathx)
df = pd.read_json(path)

result = pd.concat([df, dfx], ignore_index=True)
result


In [ ]:

#Filtro de datos
encargado = result[["RUT_ENCARGADO", "NOMBRE_ENCARGADO","APELLIDO_ENCARGADO","SEXO_ENCARGADO"]]
encargado


In [ ]:
#Eliminamos la columna que no necesitamos
encargado.drop(columns=['SEXO_ENCARGADO'], inplace=True)
encargado

In [ ]:
continente = result[["ID_CONTINENTE","NOMBRE_CONTINENTE"]]
continente


In [ ]:
#visualizar datos que son duplicados

continente[continente.duplicated()]


In [ ]:
#Borramos datos duplicados

continente.drop_duplicates(inplace=True)
continente

In [ ]:
pais = result[["ID_PAIS","NOMBRE_PAIS","ID_CONTINENTE"]]
pais

In [ ]:
ciudad = result[["ID_CIUDAD","NOMBRE_CIUDAD","ID_PAIS"]]
ciudad


In [ ]:
familia = result[["ID_FAMILIA","NOMBRE_FAMILIA"]]
familia

In [ ]:
familia[familia.duplicated()]

In [ ]:
familia["NOMBRE_FAMILIA"] = familia["NOMBRE_FAMILIA"].str.strip()
familia.drop_duplicates(inplace=True)
familia

In [ ]:
especie = result[["ID_ESPECIE","NOMBRE_ESPECIE","ID_FAMILIA"]]
especie

In [ ]:
zoologico = result[["ID_ZOOLOGICO","NOMBRE_ZOOLOGICO","TAMANO_ZOOLOGICO","PRESUPUESTO_ANUAL","RUT_ENCARGADO","ID_CIUDAD"]]
zoologico

In [ ]:
animal = result[["N_IDENTIFICACION","NOMBRE_ANIMAL","NOMBRE_CIENTIFICO","ESTATUS_CONSERVACION","SEXO_ANIMAL","ANIO_NACIMIENTO","PESO_ANIMAL","ALTURA_ANIMAL","ID_ESPECIE","ID_PAIS","ID_ZOOLOGICO"]]
animal

In [ ]:
#Eliminamos la columna que no necesitamos
animal.drop(columns=['PESO_ANIMAL','ALTURA_ANIMAL'], inplace=True)
animal

In [ ]:
pais_continente = result[["ID_CONT_PAIS","ID_PAIS","ID_CONTINENTE"]]
pais_continente

In [ ]:
encargado.to_sql('ENCARGADO', conexion, if_exists='append', index=False)
continente.to_sql('CONTINENTE', conexion, if_exists='append', index=False)
familia.to_sql('FAMILIA', conexion, if_exists='append', index=False)
pais.to_sql('PAIS', conexion, if_exists='append', index=False)
ciudad.to_sql('CIUDAD', conexion, if_exists='append', index=False)
especie.to_sql('ESPECIE', conexion, if_exists='append', index=False)
animal.to_sql('ANIMAL', conexion, if_exists='append', index=False)
zoologico.to_sql('ZOOLOGICO', conexion, if_exists='append', index=False)
pais_continente.to_sql('PAIS_CONTINENTE', conexion, if_exists='append', index=False)


In [ ]:
query = "SELECT count() as CANTIDAD FROM ANIMAL"
print("************ LA CANTIDAD DE ANIMALES RESGISTRADOS EN EL SISTEMA ES*************")
print(pd.read_sql_query(query, conexion))
print("*************************************************")

In [ ]:
print("************** Menu Sistema ************\n")
print("¨**************Zoologicos***************\n")
print("Seleccione el ID del Zoologico a consultar \n")

query_zoo = "SELECT ID_ZOOLOGICO, NOMBRE_ZOOLOGICO FROM ZOOLOGICO"

zoo = pd.read_sql_query(query_zoo, conexion)
print(zoo)
id_zoo = int(input("Ingrese ID de Zoologico"))

#resgtringir el igreso solo a numeros y no a letras ni puntos ni comas ... etc.
#isdigit()

print(f"{id_zoo}")

query_zoo= f"SELECT COUNT() FROM ZOOLOGICO WHERE ID_ZOOLOGICO = {id_zoo}"

consuzoo = pd.read_sql_query(query_zoo, conexion)
# consuzzo.iloc[x][x], se utiliza para usar el dato que existe en esa pocision del df, en este caso es para utilizar
# el resultado de la consulta SQL
idzoo = consuzoo.iloc[0][0]
index1 = 0
if(idzoo == 1):
    query_zoo = f"SELECT NOMBRE_ZOOLOGICO FROM ZOOLOGICO WHERE ID_ZOOLOGICO ={id_zoo}"
    consuzoo=pd.read_sql_query(query_zoo,conexion)
    nombre_zoo = consuzoo.iloc[0][0]
    while(index1 == 0):
        print("********************************")
        print(f"SISTEMA ZOOLOGICO {nombre_zoo}")
        #exportar los dataframe a excel
        
        print("1.- Datos del Zoologico ")#Nombre , Ciudad, Pais, Encargado, cantidad de animales
        print("2.- Animales de Zoologico") # id_animal, nombre, nombrecientifico y sexo.
        opcion = int(input("Seleccione un opcion"))
        
        
        if(opcion == 1):
                print("**** DATOS DEL ZOOLOGICO ****")
                query_zoo = f" SELECT NOMBRE_ZOOLOGICO as ZOOLOGICO, CIUDAD.NOMBRE_CIUDAD AS CIUDAD, PAIS.NOMBRE_PAIS AS PAIS, ENCARGADO.NOMBRE_ENCARGADO || ' ' || ENCARGADO.APELLIDO_ENCARGADO AS ENCARGADO, (SELECT COUNT() FROM ANIMAL WHERE ID_ZOOLOGICO  = 1006) as CANTIDAD_ANIMALES from ZOOLOGICO  join CIUDAD on CIUDAD.ID_CIUDAD = ZOOLOGICO.ID_CIUDAD JOIN PAIS ON CIUDAD.ID_PAIS = PAIS.ID_PAIS JOIN ENCARGADO ON ENCARGADO.RUT_ENCARGADO = ZOOLOGICO.RUT_ENCARGADO  WHERE ID_ZOOLOGICO = {id_zoo}"
                consuzoo=pd.read_sql_query(query_zoo,conexion) 
                print(consuzoo)
                
                print("*** DESEA EXPORTAR A EXCEL ***")
                print("1.- SI")
                print("2.- NO")
                
                op = int(input("Ingrese una opcion"))
                if(op ==1):
                        print("Vamos a exportar a excel")
                        consuzoo.to_excel("datos_zoologico.xlsx")         
                elif (op == 2):
                        
                        print("****Salir*****")
                elif(op != 1 or op != 2):
                        print("La opcion ingresada no existe\n *** SALIR DEL SISTEMA***")
                index1=index1+1   
                
                
                
        elif(opcion == 2):
                print(f"DATOS DE ANIMALES {nombre_zoo} ")
                query_zoo= f"SELECT N_IDENTIFICACION AS ID, NOMBRE_ANIMAL AS NOMBRE, NOMBRE_CIENTIFICO, SEXO_ANIMAL AS SEXO  FROM ANIMAL WHERE ID_ZOOLOGICO ={id_zoo}"
                consuzoo=pd.read_sql_query(query_zoo,conexion) 
                print(consuzoo)
                
                print("*** DESEA EXPORTAR A EXCEL ***")
                print("1.- SI")
                print("2.- NO")
                
                op = int(input("Ingrese una opcion"))
                if(op ==1):
                        print("Vamos a exportar a excel")
                        consuzoo.to_excel("datos_animales_zoologico.xlsx")         
                elif (op == 2):
                        
                        print("****Salir*****")
                elif(op != 1 or op != 2):
                        print("La opcion ingresada no existe\n *** SALIR DEL SISTEMA***")
                index1=index1+1
        
        else:
                print("No existe esa opcion")
                index1 = 0 
        print("********************************")

else:
        print("El id es incorrecto")


